### 2.01_BoyerMoore

In [1]:
# Helper functions and Boyer-Moore class
# Ref Gusfield, Dan. Algorithms on strings, trees and sequences: computer science and computational biology. Cambridge university press, 1997.

import string

def z_array(s):
    """ Use Z algorithm (Gusfield theorem 1.4.1) to preprocess s """
    assert len(s) > 1
    z = [len(s)] + [0] * (len(s)-1)
    # Initial comparison of s[1:] with prefix
    for i in range(1, len(s)):
        if s[i] == s[i-1]:
            z[1] += 1
        else:
            break
    r, l = 0, 0
    if z[1] > 0:
        r, l = z[1], 1
    for k in range(2, len(s)):
        assert z[k] == 0
        if k > r:
            # Case 1
            for i in range(k, len(s)):
                if s[i] == s[i-k]:
                    z[k] += 1
                else:
                    break
            r, l = k + z[k] - 1, k
        else:
            # Case 2
            # Calculate length of beta
            nbeta = r - k + 1
            zkp = z[k - l]
            if nbeta > zkp:
                # Case 2a: Zkp wins
                z[k] = zkp
            else:
                # Case 2b: Compare characters just past r
                nmatch = 0
                for i in range(r+1, len(s)):
                    if s[i] == s[i - k]:
                        nmatch += 1
                    else:
                        break
                l, r = k, r + nmatch
                z[k] = r - k + 1
    return z


def n_array(s):
    """ Compile the N array (Gusfield theorem 2.2.2) from the Z array """
    return z_array(s[::-1])[::-1]


def big_l_prime_array(p, n):
    """ Compile L' array (Gusfield theorem 2.2.2) using p and N array.
        L'[i] = largest index j less than n such that N[j] = |P[i:]| """
    lp = [0] * len(p)
    for j in range(len(p)-1):
        i = len(p) - n[j]
        if i < len(p):
            lp[i] = j + 1
    return lp


def big_l_array(p, lp):
    """ Compile L array (Gusfield theorem 2.2.2) using p and L' array.
        L[i] = largest index j less than n such that N[j] >= |P[i:]| """
    l = [0] * len(p)
    l[1] = lp[1]
    for i in range(2, len(p)):
        l[i] = max(l[i-1], lp[i])
    return l


def small_l_prime_array(n):
    """ Compile lp' array (Gusfield theorem 2.2.4) using N array. """
    small_lp = [0] * len(n)
    for i in range(len(n)):
        if n[i] == i+1:  # prefix matching a suffix
            small_lp[len(n)-i-1] = i+1
    for i in range(len(n)-2, -1, -1):  # "smear" them out to the left
        if small_lp[i] == 0:
            small_lp[i] = small_lp[i+1]
    return small_lp


def good_suffix_table(p):
    """ Return tables needed to apply good suffix rule. """
    n = n_array(p)
    lp = big_l_prime_array(p, n)
    return lp, big_l_array(p, lp), small_l_prime_array(n)


def good_suffix_mismatch(i, big_l_prime, small_l_prime):
    """ Given a mismatch at offset i, and given L/L' and l' arrays,
        return amount to shift as determined by good suffix rule. """
    length = len(big_l_prime)
    assert i < length
    if i == length - 1:
        return 0
    i += 1  # i points to leftmost matching position of P
    if big_l_prime[i] > 0:
        return length - big_l_prime[i]
    return length - small_l_prime[i]


def good_suffix_match(small_l_prime):
    """ Given a full match of P to T, return amount to shift as
        determined by good suffix rule. """
    return len(small_l_prime) - small_l_prime[1]


def dense_bad_char_tab(p, amap):
    """ Given pattern string and list with ordered alphabet characters, create
        and return a dense bad character table.  Table is indexed by offset
        then by character. """
    tab = []
    nxt = [0] * len(amap)
    for i in range(0, len(p)):
        c = p[i]
        assert c in amap
        tab.append(nxt[:])
        nxt[amap[c]] = i+1
    return tab


class BoyerMoore(object):
    """ Encapsulates pattern and associated Boyer-Moore preprocessing. """
    
    def __init__(self, p, alphabet='ACGT'):
        self.p = p
        self.alphabet = alphabet
        # Create map from alphabet characters to integers
        self.amap = {}
        for i in range(len(self.alphabet)):
            self.amap[self.alphabet[i]] = i
        # Make bad character rule table
        self.bad_char = dense_bad_char_tab(p, self.amap)
        # Create good suffix rule table
        _, self.big_l, self.small_l_prime = good_suffix_table(p)
    
    def bad_character_rule(self, i, c):
        """ Return # skips given by bad character rule at offset i """
        assert c in self.amap
        ci = self.amap[c]
        assert i > (self.bad_char[i][ci]-1)
        return i - (self.bad_char[i][ci]-1)
    
    def good_suffix_rule(self, i):
        """ Given a mismatch at offset i, return amount to shift
            as determined by (weak) good suffix rule. """
        length = len(self.big_l)
        assert i < length
        if i == length - 1:
            return 0
        i += 1  # i points to leftmost matching position of P
        if self.big_l[i] > 0:
            return length - self.big_l[i]
        return length - self.small_l_prime[i]
    
    def match_skip(self):
        """ Return amount to shift in case where P matches T """
        return len(self.small_l_prime) - self.small_l_prime[1]


Examples

In [2]:
# GCTAGCTCTACGAGTCTA
p = 'TCAA'
p_bm = BoyerMoore(p, alphabet='ACGT') # preprocessing
p_bm.bad_character_rule(2, 'T')

2

In [3]:
# GCTAGCTCTACGAGTCTA
# ACTA
p = 'ACTA'
p_bm = BoyerMoore(p, alphabet='ACGT')
p_bm.good_suffix_rule(0)

3

In [4]:
# ACACGCTCTACGAGTCTA
# ACAC
p = 'ACAC'
p_bm = BoyerMoore(p, alphabet='ACGT')
p_bm.match_skip()

2

In [5]:
# Implement the Boyer-Moore function:
def boyer_moore(p, p_bm, t):
    
    i = 0
    occurrences = []
    while i < len(t)-len(p)+ 1:
        shift = 1
        mismatched = False
        
        for j in range(len(p)-1, -1, -1): # note len(p)-1
            if not p[j] == t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j]) # note t[i+j] , nto p[j]
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True   # Note this step
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift 
    return occurrences
            

In [6]:
t = 'GCTAGCTCTACGAGTCTA'
p = 'TCTA'
p_bm = BoyerMoore(p, alphabet='ACGT')

In [7]:
boyer_moore(p, p_bm, t)

[6, 14]

### 2.02_SubstringIndex

In [8]:
import bisect
import sys

In [9]:
class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k # k-mer length
        self.index = []
        for i in range(len(t) - k +1):
            self.index.append((t[i:i+k],i)) # append tuples of (kmer, offset)
        self.index.sort() # sort tuples
            
    def query(self, p):
        kmer = p[:self.k] # query with first kmer. Note p[:self.k]. We just need to check one kmer, here we choose the 1st one. 
        i = bisect.bisect_left(self.index, (kmer, -1)) # binary search, get the left most offset for inserting this kmer
        hits = []
        while i < len(self.index): # search every kmer on/after offset i
            if self.index[i][0] != kmer: # if a kmer in t (on/after position i) does not match kmer of p 
                break
            hits.append(self.index[i][1]) # else, it was a match, append offset
            i += 1
        return hits

In [10]:
# Function to verify hits: characters outside of kmer
def queryIndex(p, t, index):
    k = index.k # inherited parameter from Index class
    offsets = []
    for i in index.query(p): # for every hits. Inherited function from Index class
        if p[k:] == t[i+k:i+len(p)]:  # Do not forget the 'i'!! , careful writing p[k:], not p[:k]
            offsets.append(i)
    return offsets
    

In [11]:
t = 'ACTTGGAGATCTTTGAGGCTAGGTATTCGGGATCGAAGCTCATTTCGGGGATCGATTACGATATGGTGGGTATTCGGGA'
p = 'GGTATTCGGGA'

In [12]:
index = Index(t, 4)
print(queryIndex(p,t,index))

[21, 68]


### 2.03_ApproximateMatching

In [24]:
def approximate_match(p, t, n):
    '''Apply pigeonhole principle to do approximate matching
    n = number of mismatches allowed'''
    segment_length = int(round(len(p)/(n+1)))
    all_matches = set()
    
    for i in range(n+1): # For each segment, look for matches
        start = i*segment_length # the start location of the segment
        end = min(len(p), (i+1)*segment_length)
        p_bm = BoyerMoore(p[start:end], alphabet='ACGT') # create look-up table
        matches = boyer_moore(p[start:end], p_bm, t)
        print(i, start, end,matches)
        
        for m in matches: # For each match in t, check the rest of p and see how many mismatches are there
            # NOTE: m are locations in t!
            
            if m < start or m-start+len(p)>len(t): # ?? why this condition
                continue
                
            mismatches = 0
            for j in range(0,start): # check left side of the segment in p
                print('left, j=',j,'p[j]',p[j],'t[m-start+j]',t[m-start+j])
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n: # remember to break to be efficient
                        break
            
            for j in range(end,len(p)): # check right side 
                print('right, j=',j,'p[j]',p[j],'t[m-start+j]',t[m-start+j])
                if not p[j] == t[m-start+j]: # NOTE: the slicing of t is the same as left side
                    mismatches += 1
                    if mismatches > n:
                        break
            
            if mismatches <= n:
                all_matches.add(m-start) # NOTE: use '.add', not '.append' for set()
                # NOTE: use (m-start) to store locations that match the start of p
        
    return list(all_matches) # convert to list
        
        

In [25]:
p = 'AACTTG'
t = 'CACTTAATTTG'
print(approximate_match(p, t, 2))

0 0 2 [5]
right, j= 2 p[j] C t[m-start+j] T
right, j= 3 p[j] T t[m-start+j] T
right, j= 4 p[j] T t[m-start+j] T
right, j= 5 p[j] G t[m-start+j] G
1 2 4 [2]
left, j= 0 p[j] A t[m-start+j] C
left, j= 1 p[j] A t[m-start+j] A
right, j= 4 p[j] T t[m-start+j] T
right, j= 5 p[j] G t[m-start+j] A
2 4 6 [9]
left, j= 0 p[j] A t[m-start+j] A
left, j= 1 p[j] A t[m-start+j] A
left, j= 2 p[j] C t[m-start+j] T
left, j= 3 p[j] T t[m-start+j] T
[0, 5]
